In [6]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [7]:
# MAT 파일 불러오기
mat_path = r"../HW2_package/hw2_prob2.mat"
mat_data = scipy.io.loadmat(mat_path)

# 변수 할당
A = mat_data['A']
d = mat_data['d']
d_up = mat_data['d_up']
t = 2

print(A.shape)
print(d.shape)

(3007, 1568)
(3007, 1)


In [8]:
# minimum f
def f(x):
    return 0.5 * (np.linalg.norm(np.matmul(A, x) - d, ord = 2) ** 2)
    
def f_1_differential(x):
    return A.T @ (A @ x - d)

def f_1_differential_H(x, temp_d):
    # H
    print(np.where(A @ x - temp_d > 0))
    
    A[np.where(A @ x - temp_d > 0)] = 0 
    return A.T @ (A @ x - d)

In [9]:
def F(x, t):
    return f(x) + (- np.sum(np.log(d_up - np.matmul(A, x))) / t)
    
def F_1_differential(x, t):
    Ax = A @ x
    reciprocal_terms = np.reciprocal(d_up - Ax)
    temp = A.T @ reciprocal_terms / t
    temp = temp.reshape(-1, 1)
    return f_1_differential_H(x, d_up) + temp

In [10]:
# with inequality constraint
xk = np.zeros((A.shape[1], 1))

# 조정 값
alpha = 0.01
beta = 0.8

for idx in range(10000):  
    delta_x = - F_1_differential(xk, t)
    step_size = 1
    
    #feasibility check
    while np.min(d_up- A @ (xk + step_size * delta_x)) <= 0: 
        step_size *= beta

    #backtracking line search
    while F((xk + step_size * delta_x), t) > (F(xk, t) + alpha * step_size * F_1_differential(xk, t).T @ delta_x):
        step_size *= beta
        
    xk = xk + step_size * delta_x
    
    print(idx)

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
0
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
1
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
2
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
3
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
4
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
5
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([

KeyboardInterrupt: 

In [7]:
# # DVH
# np.save(r"../HW2_package/xk_with_inequality_constraint_1.npy", xk)